In [1]:
%matplotlib inline
import os

import rasterio
import random
import json
import sys
import datacube
import matplotlib
import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
from tqdm import tqdm
from skimage.measure import find_contours
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap
from geopy.geocoders import Nominatim 
from shapely import speedups
from shapely.geometry import Point, Polygon, LineString
from shapely.ops import triangulate
from shapely.strtree import STRtree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from deafrica_tools.plotting import display_map, map_shapefile, plot_lulc, rgb
from deafrica_tools.datahandling import load_ard, mostcommon_crs
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.coastal import tidal_tag
from deafrica_tools.dask import create_local_dask_cluster
from datacube.utils.geometry import Geometry
from deafrica_tools.spatial import xr_rasterize

import matplotlib.colors as mcolors

In [2]:
create_local_dask_cluster()

/usr/local/lib/python3.10/dist-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43059 instead
  warnings.warn(
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:37311
INFO:distributed.scheduler:  dashboard at:           127.0.0.1:43059
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:39117'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:39555', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:39555
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:46360
INFO:distributed.scheduler:Receive client connection: Client-aa54244b-2fc5-11ef-82ca-b65b30182ddd
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:46376


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/ndeye.fatou.sane28@gmail.com/proxy/43059/status,
Dashboard: /user/ndeye.fatou.sane28@gmail.com/proxy/43059/status,Workers: 1
Total threads: 15,Total memory: 97.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37311,Workers: 1
Dashboard: /user/ndeye.fatou.sane28@gmail.com/proxy/43059/status,Total threads: 15
Started: Just now,Total memory: 97.21 GiB
Comm: tcp://127.0.0.1:39555,Total threads: 15
Dashboard: /user/ndeye.fatou.sane28@gmail.com/proxy/44377/status,Memory: 97.21 GiB
Nanny: tcp://127.0.0.1:39117,


In [3]:
dc = datacube.Datacube(app="Monitoring_seagrass")

In [4]:
# Lire le fichier GeoJSON et définir le CRS
AMPs_SN = gpd.read_file("AMP_SN_2023.geojson").set_crs(32628)

# Filtrer le GeoDataFrame pour sélectionner 'Somone'
idx = AMPs_SN[AMPs_SN['APNAME'] == "Somone"].index[0]
geom = Geometry(geom=AMPs_SN.iloc[idx].geometry, crs=AMPs_SN.crs)

gdf = AMPs_SN[AMPs_SN['APNAME'] == "Somone"].set_crs(32628)

# Visualiser le shapefile avec 'APNAME' comme attribut
map_shapefile(gdf, attribute='APNAME')

Label(value='')

Map(center=[14.501419633643806, -17.09514418085054], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [5]:
time = ("2023-01", "2023-03") 
sample_frequency="5D" 
#tide_range = (0.25, 0.75)  # tide_range = (0.25, 0.75)

In [ ]:
# Create a reusable query

query = {
    "geopolygon": geom,
    "resolution": (-10, 10),
    "output_crs":"EPSG:32628",
    "group_by": "solar_day",
    "time": time,
     
    }
# "geopolygon": geom,

# Load available data from Sentinel-2A and -2B and filter to retain only times
# with at least 80% good data
sentinel_2_ds = load_ard(dc=dc, 
              products=["s2_l2a"],
              min_gooddata=0.9,
              measurements =["red_edge_1", "red","green","blue","nir"],      
              
              **query)
# min_gooddata=0.9,

In [ ]:
#sentinel_2_ds

In [ ]:
#sentinel_2_ds=sentinel_2_ds.sel(time='2023-02-16')

In [ ]:
rgb(sentinel_2_ds,col='time')

In [ ]:
# Set the timesteps to visualise
#timestep = 5

# Generate RGB plots at each timestep
#rgb(sentinel_2_ds, 
   #index=timestep,
   # percentile_stretch=(0.05, 0.95))

In [ ]:
# Set the timesteps to visualise
#initial_timestep = 1
#final_timestep = 25

# Generate RGB plots at each timestep
#rgb(sentinel_2_ds, index=[initial_timestep, final_timestep],
   #percentile_stretch=[0.01, 0.99])

## Compute band indices
This study measures the presence of water through the normalised difference water index (NDWI), submerged seagrasses through the Submerged Seagrasses Identification Index (SSII) and the non submerged seagrasses through the normalised difference vegetation index (NDVI).

NDWI is calculated from the green and near infrared (NIR) bands to identify water.
The formula is

$$
\begin{aligned}
\text{NDWI} = \frac{\text{Green} - \text{NIR}}{\text{Green} + \text{NIR}}.
\end{aligned}
$$

NDVI is calculated from the red and near infrared (NIR) bands to identify non submerged seagrasses.
The formula is

$$
\begin{aligned}
\text{NDVI} = \frac{\text{Red} - \text{NIR}}{\text{Red} + \text{NIR}}.
\end{aligned}
$$

SSII is calculated from the  first vegetation red edge band, and a is the adjustment factor, which takes a very small value to 
ensure that the denominator is not 0. The value of 0.00001 was taken in the experiment described in this study. [Li and all (2023)](https://doi.org/10.1016/j.marenvres.2023.105880)
The formula is

$$
\begin{aligned}
\text{SSII} = \frac{\text{Red edge 1} }{\text{Red} + \text{a}}.
\end{aligned}
$$

In [ ]:
# Calculate MNDWI and add it to the loaded data set
sentinel_2_ds = calculate_indices(sentinel_2_ds, index="NDWI", collection="s2")

# Calculate NDVI and add it to the loaded data set
sentinel_2_ds = calculate_indices(sentinel_2_ds, index="NDVI", collection="s2")

In [ ]:
water_mask = sentinel_2_ds.NDWI > 0.0

In [ ]:
# Plot the resulting image for the same timestep selected above
#timestep = 9
#sentinel_2_ds.NDWI.isel(time=timestep).plot(cmap='RdBu',
                                          #size=6,
                                          #vmin=-0.5,
                                         # vmax=0.5)
#plt.show()

In [ ]:
# Plot the resulting image for the same timestep selected above
sentinel_2_ds.NDWI.plot(col="time", cmap='RdBu',
                                         size=6,
                                          vmin=-0.5,
                                          vmax=0.5)
plt.show()

In [ ]:
# Liste pour stocker les lignes de bord extraites
water_edges = []

# Itérer sur chaque pas de temps dans les données Sentinel-2
for time_step in sentinel_2_ds.time:
    # Sélectionner l'image NDWI pour le pas de temps courant
    ndwi_image = sentinel_2_ds.NDWI.sel(time=time_step)

    # Appliquer le masque d'eau
    water_ndwi = ndwi_image.where(water_mask.sel(time=time_step))

    # Trouver les contours des zones d'eau
    contours = find_contours(water_ndwi.values, level=0.0)

    # Convertir les contours en géométries Shapely et les ajouter à la liste
    for contour in contours:
        line = LineString(contour)
        water_edges.append(gpd.GeoDataFrame({'geometry': [line], 'time': [time_step.values]}, crs=sentinel_2_ds.crs))

In [ ]:
# Combiner les lignes de bord en un seul GeoDataFrame
water_edges_gdf = gpd.GeoDataFrame(pd.concat(water_edges, ignore_index=True))


In [ ]:
# Calculate tides for each timestep in the satellite dataset
sentinel_2_ds = tidal_tag(ds=sentinel_2_ds, tidepost_lat=None, tidepost_lon=None)

# Print the output dataset with new `tide_height` variable
print(sentinel_2_ds)


In [ ]:
# Calculate the min and max tide heights to include based on the % range
min_tide, max_tide = sentinel_2_ds.tide_m.quantile(tide_range) 

# Plot the resulting tide heights for each Landsat image:
sentinel_2_ds.tide_m.plot()
plt.axhline(min_tide, c='red', linestyle='--')
plt.axhline(max_tide, c='red', linestyle='--')
plt.show()


In [ ]:
import os
import numpy as np
import rasterio
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import geopandas as gpd

# 1. Téléchargement des Images Sentinel-2
def download_sentinel_data(username, password, footprint, start_date, end_date):
    api = SentinelAPI(username, password, 'https://scihub.copernicus.eu/dhus')
    products = api.query(footprint,
                         date=(start_date, end_date),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))
    api.download_all(products)

# 2. Prétraitement des Images
def preprocess_image(image_path):
    with rasterio.open(image_path) as src:
        band_red = src.read(3)
        band_green = src.read(2)
        band_blue = src.read(1)
        band_nir = src.read(8)

        # Normalisation
        band_red = band_red / 10000.0
        band_green = band_green / 10000.0
        band_blue = band_blue / 10000.0
        band_nir = band_nir / 10000.0
        
        return np.stack([band_red, band_green, band_blue, band_nir], axis=-1)

# 3. Classification
def classify_image(image):
    # Placeholder for training data and labels
    X = []  # Features
    y = []  # Labels
    
    # Add your training data collection logic here

    # Example: Train a RandomForestClassifier
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_train)
    
    prediction = classifier.predict(image.reshape(-1, image.shape[2])).reshape(image.shape[0], image.shape[1])
    return prediction

# 4. Visualisation
def plot_classification(prediction, image_path):
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))
    with rasterio.open(image_path) as src:
        ax.imshow(src.read([3, 2, 1]).transpose(1, 2, 0) / 10000.0, extent=src.bounds, cmap='gray')
        ax.imshow(prediction, alpha=0.5, cmap='viridis')
    plt.show()

# Exemple d'utilisation
username = 'votre_nom_utilisateur'
password = 'votre_mot_de_passe'
footprint = geojson_to_wkt(read_geojson('votre_fichier_geojson.geojson'))
start_date = '20220101'
end_date = '20220131'

# Télécharger les données
download_sentinel_data(username, password, footprint, start_date, end_date)

# Prétraiter une image
image_path = 'chemin/vers/votre/image.tif'
image = preprocess_image(image_path)

# Classifier l'image
prediction = classify_image(image)

# Visualiser le résultat
plot_classification(prediction, image_path)
